In [13]:
from database.models import Workflow, WorkflowVersion

In [14]:
import boto3
from botocore.exceptions import ClientError

from urllib.parse import urlparse
from tempfile import NamedTemporaryFile

In [15]:
TEST_USER_ID = "111"
TEST_COLLECTION_ID = "444"


In [16]:
from platformics.database.connect import init_sync_db

In [17]:
def SYNC_DB_URI() -> str:
    db_uri = "postgresql+psycopg://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}".format(
        db_host="localhost",
        db_port=5432,
        db_user="postgres",
        db_pass="password_postgres",
        db_name="workflows",
    )
    return db_uri

sync_db_uri = SYNC_DB_URI()

In [18]:
app_db = init_sync_db(sync_db_uri)
session = app_db.session()

In [19]:
cg_workflow = session.query(Workflow).filter_by(name="consensus-genome").first() or Workflow()
cg_workflow.owner_user_id = TEST_USER_ID
cg_workflow.collection_id = TEST_COLLECTION_ID
cg_workflow.default_version = "3.4.18"
cg_workflow.minimum_supported_version = "3.4.18"

session.add(cg_workflow)
session.commit()

In [20]:
local_s3 = boto3.client("s3", endpoint_url="http://localhost:4000", region_name="us-east-1")
real_s3 = boto3.client("s3")

local_s3.upload_file("/home/todd/czid-workflows/workflows/consensus-genome/run.wdl", "local-bucket", "consensus-genome-v3.4.18/run.wdl")
s3_path = "s3://local-bucket/consensus-genome-v3.4.18/run.wdl"

def transfer(s3_path: str):
    parsed = urlparse(s3_path)
    bucket, key = parsed.netloc, parsed.path[1:]
    try:
        local_s3.create_bucket(Bucket=bucket)
    except ClientError as e:
        if e.response["Error"]["Code"] == "BucketAlreadyOwnedByYou":
            pass
        elif e.response["Error"]["Code"] == "BucketAlreadyExists":
            pass
        else:
            raise e

    with NamedTemporaryFile() as f:
        real_s3.download_file(bucket, key, f.name)
        local_s3.upload_file(f.name, bucket, key)

In [21]:
local_s3.upload_file("/home/todd/czid-workflows/workflows/consensus-genome/integration_test/sample_sars-cov-2_paired_r1.fastq.gz", "local-bucket", "sample_sars-cov-2_paired_r1.fastq.gz")
local_s3.upload_file("/home/todd/czid-workflows/workflows/consensus-genome/integration_test/sample_sars-cov-2_paired_r2.fastq.gz", "local-bucket", "sample_sars-cov-2_paired_r2.fastq.gz")

In [22]:
with open("/home/todd/czid-workflows/workflows/consensus-genome/manifest_simple.yml") as f:
    manifest_str = f.read()

In [23]:
cg_workflow_version = WorkflowVersion(
    owner_user_id=TEST_USER_ID,
    collection_id=TEST_COLLECTION_ID,
    graph_json="{}",
    workflow=cg_workflow,
    manifest=manifest_str,
    workflow_uri=s3_path,

)
session.add(cg_workflow_version)
session.commit()

In [28]:
with open("/home/todd/czid-workflows/workflows/consensus-genome/manifest.yml") as f:
    manifest_str = f.read()
cg_workflow_version.manifest = manifest_str
session.add(cg_workflow_version)
session.commit()

In [27]:
cg_workflow_version.id

UUID('018d9ad0-e8da-7224-9e65-8016b4fb8b85')

In [ ]:
        return {
          "fastqs_0": "/home/todd/czid-workflows/workflows/consensus-genome/integration_test/blank.fastq.gz",
          "sample": "test_sample",
          "technology": "ONT"
        }